### 方法论

#### 3.1 问题定义

**FS-PCS（少样本点云分割）**  
这一任务可以被表示为流行的**逐集范式**（episodic paradigm）（Vinyals 等，2016），并延续了以往的研究工作（Zhao 等，2021；An 等，2024）。每个集对应一个 $ N $-分类 $ K $-次分割任务，包括一个支持集 $\mathcal{S} = \{[\mathbf{X}_s^{n,k}, \mathbf{Y}_s^{n,k}]_{k=1}^K\}_{n=1}^N$ 和一个查询集 $\mathcal{Q} = \{\mathbf{X}_q^n, \mathbf{Y}_q^n\}_{n=1}^N$。我们用 $\mathbf{X}_{s/q}$ 和 $\mathbf{Y}_{s/q}$ 分别表示点云和其对应的点级别标签。支持集 $\mathcal{S}$ 包含 $ N $ 个目标类别的样本，每个类别 $ n $ 由一个 $ K $-次样本组 $\{\mathbf{X}^{n,k}, \mathbf{Y}^{n,k}\}_{k=1}^K$ 描述，且每组样本仅包含对应语义类别的标签。FS-PCS 的目标是利用支持集中 $ N $ 个新类别 $\mathcal{S}$ 的知识，将查询样本 $\{\mathbf{X}_q^n\}_{n=1}^N$ 分割为 $ N $ 个目标类别和“背景”类别。

**多模态 FS-PCS**  
与现有设置不同，我们提出了一个多模态 FS-PCS 设置，其中包含两个额外的模态：文本模态和 2D 图像模态。具体而言，对于上面介绍的逐集范式，每个支持集类别 $ \mathcal{S} $ 除了点云数据，还附带 $ N $ 个类别名称（例如，`chair`，`table`，`wall` 等）。对于 2D 图像模态，在预训练阶段，3D 点云数据伴随有 2D RGB 图像，但在元学习和推理阶段并不需要 2D 图像数据。在接下来的讨论中，除非另有说明，我们将专注于 1-分类 1-次（1-way 1-shot）设置以便于说明。支持集和查询集分别表示为 $\mathcal{S} = \{\mathbf{X}_s, \mathbf{Y}_s\}$ 和 $\mathcal{Q} = \{\mathbf{X}_q, \mathbf{Y}_q\}$。

### 3.2 方法概述

#### 我们的想法 

由于现有的少样本点云分割（FS-PCS）数据集包含三种模态（3D 点云、类别名称和 2D RGB 图像），且这些数据集规模较小，直接训练模型来学习这些模态的有意义表示十分困难。受到视觉-语言模型（Vision-Language Models, VLMs）快速发展的启发，我们提出利用现有的 VLM（如 LSeg (Li 等，2022) 和 OpenSeg (Ghiasi 等，2022)）来为 FS-PCS 引入额外的模态支持。  
具体来说，我们采用预训练的 LSeg 文本编码器提取类别名称的文本嵌入。这些强大的文本嵌入为 FS-PCS 的学习提供了额外的指导，作为从支持集中提取的视觉信息的补充。为了利用可能可用的 2D 模态，我们提议使用 LSeg 的视觉编码器生成 2D 视觉特征。由于 LSeg 模型基于大规模 2D 数据集进行预训练，这些视觉特征具有出色的泛化能力。  
考虑到并非所有 FS-PCS 数据集（如 Armeni 等，2016）都包含 2D 模态，我们在预训练阶段利用提取的 2D 特征来监督 3D 点云特征的学习，从而有效地通过 3D 特征模拟 2D 特征。学习到的特征被称为**模态交互特征（intermodal features）**，因为它们同时包含 3D 和 2D 信息。这一设计具有两大优势：  
1. 我们的模型以隐式方式使用 2D 模态，无需在元学习和推理阶段作为输入。  
2. 学到的模态交互特征与 LSeg 的 2D 视觉特征对齐，因此也与文本嵌入对齐。这种对齐为后续阶段提供了重要的指导，后续部分将对此进行详细解释。  

#### 方法概述
 
所提出的 MM-FSS 的整体架构如图 2 所示。  
给定支持点云和查询点云，我们首先生成两组高级特征：通过模态交互特征头（Intermodal Feature, IF 头）提取的**模态交互特征**，以及通过单模态特征头（Unimodal Feature, UF 头）提取的点云模态的**单模态特征**。模态交互特征和单模态特征随后输入多模态相关性融合模块（Multimodal Correlation Fusion, MCF），以生成支持点云和查询点云之间的多模态相关性。  
在挖掘视觉连接之外，我们利用 LSeg 文本编码器（Li 等，2022）为类别名称生成文本嵌入。然后，我们在多模态语义融合模块（Multimodal Semantic Fusion, MSF）中，利用文本模态提供的有用语义指导来进一步优化多模态相关性。在推理阶段，为了减轻训练偏差（Cheng 等，2022），我们进一步提出**测试时自适应跨模态校准（Test-time Adaptive Cross-modal Calibration, TACC）**，以生成针对新类别的更好预测。  
现有的 FS-PCS 方法（An 等，2024；Zhao 等，2021）通常包括两个训练步骤：一个用于获得有效特征提取器的预训练步骤，以及一个面向少样本分割任务的元学习步骤。我们的方法遵循这一两步训练范式。首先，我们通过 3D 点云和 2D 图像预训练骨干网络和 IF 头；其次，在冻结骨干网络和 IF 头的同时，进行端到端的元学习训练。更多训练细节见附录 B。  
在接下来的部分中，我们将详细介绍特征提取器（3D 骨干网络、IF 和 UF 头、文本编码器）以及 MCF、MSF 和 TACC 模块。

### 3.3 特征提取器

#### 视觉特征

我们的方法通过联合骨干网络和两个不同的头部（IF和UF）处理点云输入，如图2所示。IF头部通过利用2D模态提取与2D视觉特征对齐的跨模态特征，而UF头部则专注于3D点云模态。给定支持/查询点云 $\mathbf{X}_{s/q}$，我们使用共享骨干网络 $\Phi$ 来获得通用支持特征 $\mathbf{F}_s = \Phi(\mathbf{X}_s) \in \mathbb{R}^{N_S \times D}$ 和查询特征 $\mathbf{F}_q = \Phi(\mathbf{X}_q) \in \mathbb{R}^{N_Q \times D}$，其中 $D$ 是通道维度，$N_S$ 和 $N_Q$ 分别是 $\mathbf{X}_s$ 和 $\mathbf{X}_q$ 中的点数量。随后，这些特征由 IF 头部（$\mathcal{H}_{\text{IF}}$）和 UF 头部（$\mathcal{H}_{\text{UF}}$）处理，为支持和查询输入生成跨模态和单模态特征，表示为：

$$
\begin{aligned}
\mathbf{F}_s^{\text{i}} &= \mathcal{H}_{\text{IF}}(\mathbf{F}_s) \in \mathbb{R}^{N_S \times D_{\text{i}}}, \quad \mathbf{F}_s^{\text{u}} = \mathcal{H}_{\text{UF}}(\mathbf{F}_s) \in \mathbb{R}^{N_S \times D}, \\
\mathbf{F}_q^{\text{i}} &= \mathcal{H}_{\text{IF}}(\mathbf{F}_q) \in \mathbb{R}^{N_Q \times D_{\text{i}}}, \quad \mathbf{F}_q^{\text{u}} = \mathcal{H}_{\text{UF}}(\mathbf{F}_q) \in \mathbb{R}^{N_Q \times D}.
\end{aligned}
$$

$D_{\text{i}}$ 表示跨模态特征的通道维度，与 LSeg（Li等人，2022）的嵌入空间对齐。所得到的 $\mathbf{F}_s^{\text{i}}$ 和 $\mathbf{F}_s^{\text{u}}$ 分别表示支持点云的跨模态和单模态特征。$\mathbf{F}_q^{\text{i}}$ 和 $\mathbf{F}_q^{\text{u}}$ 则为查询点云提供相同的功能。

如上所述，跨模态特征 $\mathbf{F}_s^{\text{i}}$ 和 $\mathbf{F}_q^{\text{i}}$ 在第一步中经过专门训练，以与 VLMs 的视觉编码器的 2D 视觉特征对齐（Li等人，2022；Ghiasi等人，2022）。参照 Peng 等人（2023）的做法，我们使用余弦相似度损失来最小化 3D 点跨模态特征与相应 2D 像素特征之间的距离（见附录B）。完成此步骤后，我们固定骨干网络和IF头部以保持跨模态特征，为 FS-PCS 提供关键指导。然后，我们开始元学习，对整个模型进行端到端训练，以充分利用跨模态和单模态特征以及文本嵌入来进行 FS-PCS。

#### 文本嵌入

我们使用LSeg（Li等人，2022）文本编码器计算"背景"和目标类的嵌入，表示为 $\mathbf{T} = \{\mathbf{t}_0, \cdots, \mathbf{t}_N\} \in \mathbb{R}^{N_C \times D_{\text{i}}}$，其中 $\mathbf{t}_0$ 表示"背景"类的文本嵌入，其他则表示目标类的文本嵌入。这里，$N_C = N + 1$ 表示N-way设置中所有类别的数量。

### 3.4 跨模态信息融合

我们已经获得了支持/查询点云的跨模态和单模态特征，以及目标类别的文本嵌入。我们的目标是利用来自不同模态的所有可用信息来预测查询点云的分割掩码。如Min等人（2021）、Hong等人（2022）和An等人（2024）所述，小样本分割的核心是在查询和支持点云之间建立信息丰富的相关性。为此，我们提出了两个用于跨模态知识融合的新型模块：MCF和MSF。前者整合跨模态和单模态特征以生成多模态相关性，后者利用文本语义指导进一步细化这些相关性。这两个模块的详细内容如下所述。

#### 多模态相关性融合

与仅依赖单模态输入的传统FS-PCS模型（Zhao等人，2021；He等人，2023；An等人，2024）不同，我们的方法通过整合来自跨模态和单模态特征的两种相关性来计算多模态相关性。首先，使用最远点采样和点到样本聚类（如An等人（2024）和Zhao等人（2021）所述）从标注的支持点生成前景和背景原型，用于 $\mathbf{F}_s^{\text{i}}$ 和 $\mathbf{F}_s^{\text{u}}$。这个原型生成过程表示为 $\mathcal{F}_{\text{proto}}$，结果为：

$$
\begin{aligned}
\mathbf{P}_{\text{fg}}^{\text{i}}, \mathbf{P}_{\text{bg}}^{\text{i}} &= \mathcal{F}_{\text{proto}}(\mathbf{F}_s^{\text{i}}, \mathbf{Y}_s, \mathbf{L}_s), \quad \mathbf{P}_{\text{fg}}^{\text{i}}, \mathbf{P}_{\text{bg}}^{\text{i}} \in \mathbb{R}^{N_P \times D_{\text{t}}}, \\
\mathbf{P}_{\text{fg}}^{\text{u}}, \mathbf{P}_{\text{bg}}^{\text{u}} &= \mathcal{F}_{\text{proto}}(\mathbf{F}_s^{\text{u}}, \mathbf{Y}_s, \mathbf{L}_s), \quad \mathbf{P}_{\text{fg}}^{\text{u}}, \mathbf{P}_{\text{bg}}^{\text{u}} \in \mathbb{R}^{N_P \times D},
\end{aligned}
$$

其中 $\mathbf{L}_s$ 表示支持点的3D坐标，$N_P$ 是原型的数量。这些原型随后被连接得到：$\mathbf{P}_{\text{proto}}^{\text{i}} = \mathbf{P}_{\text{fg}}^{\text{i}} \oplus \mathbf{P}_{\text{bg}}^{\text{i}} \in \mathbb{R}^{(N_C \times N_P) \times D_{\text{t}}}$ 和 $\mathbf{P}_{\text{proto}}^{\text{u}} = \mathbf{P}_{\text{fg}}^{\text{u}} \oplus \mathbf{P}_{\text{bg}}^{\text{u}} \in \mathbb{R}^{(N_C \times N_P) \times D}$。随后，我们计算查询点和这些原型之间的相关性：

$$
\mathbf{C}^{\text{i}} = \frac{\mathbf{F}_q^{\text{i}} \cdot (\mathbf{P}_{\text{proto}}^{\text{i}})^{\top}}{\|\mathbf{F}_q^{\text{i}}\| \|\mathbf{P}_{\text{proto}}^{\text{i}}\|}, \quad \mathbf{C}^{\text{u}} = \frac{\mathbf{F}_q^{\text{u}} \cdot (\mathbf{P}_{\text{proto}}^{\text{u}})^{\top}}{\|\mathbf{F}_q^{\text{u}}\| \|\mathbf{P}_{\text{proto}}^{\text{u}}\|}
$$

得到 $\mathbf{C}^{\text{i}} \in \mathbb{R}^{N_Q \times (N_C \times N_P)}$ 和 $\mathbf{C}^{\text{u}} \in \mathbb{R}^{N_Q \times (N_C \times N_P)}$，它们分别表示查询点和支持原型在跨模态和单模态特征空间中的点-类别关系。这个过程在图2中被称为相关性生成。接下来，我们的MCF模块使用两个线性层转换这些相关性，然后将它们组合以获得聚合的多模态相关性 $\mathbf{C}_0$，如下所示：

$$
\mathbf{C}_0 = \mathcal{F}_{\text{lin}}(\mathbf{C}^{\text{i}}) + \mathcal{F}_{\text{lin}}(\mathbf{C}^{\text{u}}), \quad \mathbf{C}_0 \in \mathbb{R}^{N_Q \times N_C \times D},
$$

其中 $\mathcal{F}_{\text{lin}}$ 表示将 $N_P$ 通道在 $\mathbf{C}^{\text{i/u}}$ 中投影到 $D$ 的线性层。MCF模块有效地聚合了由不同模态信息的点到原型关系，通过查询点和支持类之间连接的全面多模态理解来增强相关性 $\mathbf{C}_0$。这种丰富的理解有助于从支持点云到查询点云的知识迁移，改善查询分割效果。

#### 多模态语义融合

虽然MCF模块有效地融合了来自不同信息源的相关性，但文本嵌入的语义信息仍未被利用，而这些信息可以提供有价值的语义指导来改善相关性。因此，我们提出了MSF模块，如图2所示。MSF整合了文本嵌入的语义信息来细化MCF的相关性输出。另外，由于视觉和文本模态在不同点和类别之间的相对重要性各不相同（Yin等人，2021；Cheng等人，2021），MSF动态地为每个查询点和目标类的文本语义指导分配不同的权重，以考虑模态间的不同重要性。

给定文本嵌入 $\mathbf{T}$ 和查询点云的跨模态特征 $\mathbf{F}_q^{\text{i}}$，由于跨模态特征 $\mathbf{F}_q^{\text{i}}$ 经过预训练以模拟来自VLMs的2D视觉特征（Li等人，2022），$\mathbf{F}_q^{\text{i}}$ 与文本嵌入 $\mathbf{T}$ 良好对齐，它们之间的亲和度为如何将查询点与目标类关联提供了信息丰富的指导。因此，我们首先计算查询跨模态特征和文本嵌入之间的相似度，以生成语义指导 $\mathbf{G}_q \in \mathbb{R}^{N_Q \times N_C}$ 用于目标类的分割，计算公式为：

$$
\mathbf{G}_q = \mathbf{F}_q^{\text{i}} \cdot \mathbf{T}^{\top}
$$

然后，我们的MSF模块由 $K$ 个MSF块组成，其中当前块的相关性输入表示为 $\mathbf{C}_k$ ($k \in \{0,1,\cdots,K-1\}$)。在每个块中，点-类别权重被动态计算以考虑视觉和文本模态之间的不同重要性，计算如下：

$$
\mathbf{W}_q = \mathcal{F}_{\text{mlp}}(\mathcal{F}_{\text{expand}}(\mathbf{G}_q) \oplus \mathbf{C}_k), \quad \mathbf{W}_q \in \mathbb{R}^{N_Q \times N_C \times 1}
$$

其中 $\mathcal{F}_{\text{expand}}$ 在 $\mathbf{G}_q$ 的最后维度上进行扩展和重复，将其转换为 $\mathbb{R}^{N_Q \times N_C \times D}$，$\mathcal{F}_{\text{mlp}}$ 表示多层感知机（MLP）。接下来，语义指导 $\mathbf{G}_q$ 经过 $\mathbf{W}_q$ 加权后，被聚合到相关性输入 $\mathbf{C}_k$ 中。使用线性注意力层（Katharopoulos等人，2020）和MLP层进一步细化相关性，表示为：

$$
\begin{aligned}
\mathbf{C}_k' &= \mathbf{G}_q \odot \mathbf{W}_q + \mathbf{C}_k, \\
\mathbf{C}_{k+1} &= \mathcal{F}_{\text{mlp}}(\mathcal{F}_{\text{attention}}(\mathbf{C}_k')),
\end{aligned}
$$

其中 $\odot$ 表示Hadamard积，$\mathcal{F}_{\text{attention}}$ 表示线性注意力层。注意，为简化起见，此处省略了 $\mathcal{F}_{\text{attention}}$ 和 $\mathcal{F}_{\text{mlp}}$ 之后的残差连接。

此MSF模块充分利用文本模态的有用语义信息来增强查询和支持点云之间的相关性，帮助确定查询点的最佳类别。值得注意的是，它计算了所有点和类别对之间视觉和文本模态的相对重要性，提高了文本模态的有效整合。

#### 损失函数

经过具有 $K$ 个块的MSF模块后，优化后的相关性 $\mathbf{C}_K$ 通过一个解码器转换为预测 $\mathbf{P}_q \in \mathbb{R}^{N_Q \times N_C}$，该解码器由KPConv（Thomas等人，2019）层和MLP层组成。整个模型通过计算预测 $\mathbf{P}_q$ 和查询点云的真实掩码 $\mathbf{Y}_q$ 之间的交叉熵损失进行端到端优化。

### 3.5 测试时自适应跨模态校准

小样本模型在训练期间对基础类别进行完全监督，不可避免地会对这些类别产生偏差（Lang等人，2022；Cheng等人，2022；Wang等人，2023；An等人，2024）。当小样本模型在新类别上进行评估时，这种基础偏差会导致测试场景中存在的基础类别出现错误激活，影响泛化能力。

为了缓解这个问题，我们提出了一个简单但有效的TACC模块，该模块仅在测试时使用。TACC模块利用语义指导 $\mathbf{G}_q$ 来校准预测 $\mathbf{P}_q$。值得注意的是，$\mathbf{G}_q$ 是从查询跨模态特征和文本嵌入中得出的，这些特征和嵌入在整个元学习过程中都不会更新。因此，$\mathbf{G}_q$ 包含的对训练类别的偏差要少得多。此外，$\mathbf{G}_q$ 包含查询点云的丰富语义信息，其中 $\mathbf{G}_q[i,:]$ 表示将第 $i$ 个点分配给目标类别的概率。基于此，我们提出通过自适应指标 $\gamma$ 对 $\mathbf{G}_q$ 和 $\mathbf{P}_q$ 进行自适应组合，实现 $\mathbf{G}_q$ 中语义信息的适当利用，得到最终预测：

$$
\hat{\mathbf{P}}_q = \gamma\mathbf{G}_q + \mathbf{P}_q
$$

这里，$\gamma$ 是一个自适应指标，反映了 $\mathbf{G}_q$ 中包含的语义质量。如果 $\gamma$ 值较高，表示 $\mathbf{G}_q$ 的质量良好，将使用更多 $\mathbf{G}_q$ 中的信息。如果 $\gamma$ 值较低，表示 $\mathbf{G}_q$ 的质量不理想，将使用较少 $\mathbf{G}_q$ 中的信息。

#### 自适应指标

提出的自适应指标 $\gamma$ 是通过评估支持样本的 $\mathbf{G}_s$ 为每个小样本场景动态计算的。使用支持跨模态特征 $\mathbf{F}_s^{\text{i}}$ 和文本嵌入 $\mathbf{T}$，我们计算 $\mathbf{G}_s$，然后用它来生成预测标签 $\mathbf{P}_s$。通过将包含已知支持标签 $\mathbf{Y}_s$ 的每个场景中 $\mathbf{G}_s$ 的质量与预测标签 $\mathbf{P}_s$ 比较，使用交并比（IoU）分数进行量化。由于 $\mathbf{G}_q$ 和 $\mathbf{G}_s$ 是使用相同的跨模态特征和文本嵌入计算的，这个分数可以作为 $\gamma$ 使用，表示 $\mathbf{G}_q$ 中语义指导的可靠性：

$$
\gamma = \frac{\sum_i \mathbf{1}_{\{\mathbf{P}_s(i)=1 \wedge \mathbf{Y}_s(i)=1\}}}{\sum_i \mathbf{1}_{\{\mathbf{P}_s(i)=1 \vee \mathbf{Y}_s(i)=1\}}}, \quad \mathbf{P}_s[i] = \arg\max(\mathbf{G}_s[i,:]), \quad \mathbf{G}_s = \mathbf{F}_s^{\text{i}} \cdot \mathbf{T}^{\top}
$$

其中 $\mathbf{1}_{\{x\}}$ 是指示函数，当x为真时等于1，否则等于0，$\mathbf{P}_s[i]$ 表示第 $i$ 个支持点的预测类别索引。这个自适应指标确保TACC模块通过在测试时动态校准预测来有效缓解训练偏差，从而提高小样本泛化能力。

----

## 附录 B 实现细节

### 训练策略
我们提供更多关于训练策略的细节。我们提出的模型被设计为具有共享相同骨干网络的双头统一架构。交互式特征(IF)头生成跨模态特征，而单模态特征(UF)头仅专注于点云模态的特征。提取信息丰富的交互式特征和单模态特征的有效训练对于实现最佳性能至关重要。由于不同模态之间存在显著的异质性和不同的监督目标，同时训练两个头部可能会使优化过程复杂化并造成不稳定。此外，现有的跨模态模型通常在标准范式下进行训练，将此类跨模态对齐学习转移到IF头部的情景训练范式中可能会影响性能。因此，我们采用两步训练策略来缓解潜在的性能问题。

### 预训练细节
在第一步中，我们专注于训练IF头部以学习与2D模态对齐的稳健3D特征，为后续的情景训练提供坚实的基础。具体来说，给定点云中点 $\mathbf{p} \in \mathbb{R}^3$ 的3D坐标和场景的分辨率为 $H \times W$ 的RGB图像 $\mathbf{I}$，我们通过投影 $\mathbf{\tilde{u}} = M_{int} \cdot M_{ext} \cdot \mathbf{p}$ 将3D点 $\mathbf{p}$ 与图像平面上对应的2D像素 $\mathbf{u} = (u,v)$ 对齐，其中 $M_{int}$ 是相机到像素的内参矩阵，$M_{ext}$ 是世界到相机的外参矩阵，$\mathbf{\tilde{u}}$ 和 $\mathbf{p}$ 分别是 $\mathbf{u}$ 和 $\mathbf{p}$ 的齐次坐标。

与文本模态对齐的2D特征 $\mathbf{F}_{2d} \in \mathbb{R}^{H \times W \times D_t}$ 可以使用LSeg中的预训练图像编码器或其他VLMs提取，而具有 $M$ 个点的点云的3D特征 $\mathbf{F}_{3D} \in \mathbb{R}^{M \times D_t}$ 由IF头部生成。然后，对于来自2D-3D对应关系的匹配3D点和2D像素，我们使用余弦相似度损失来优化骨干网络和IF头部，以确保来自 $\mathbf{F}_{3D}$ 的3D点特征与其在 $\mathbf{F}_{2d}$ 中配对的2D像素特征之间的紧密对齐，这遵循了(Peng et al. (2023))的方法。

一旦IF头部和骨干网络训练完成，它们在后续的情景训练阶段将被冻结，以维持学习到的交互式特征的完整性。因此，我们确保IF头部的表达性交互式特征得以保留，并为情景训练期间我们提出的融合模块中的跨模态集成做好准备。

对于像ScanNet (Dai et al., 2017) 这样提供2D图像和相机矩阵的数据集，直接特征对齐是可行的。对于没有2D图像的数据集，如S3DIS (Armeni et al., 2016)，我们可以直接使用从ScanNet预训练的IF头部和骨干网络。预训练步骤是与VLMs的嵌入空间对齐而无需使用任何语义标签，使预训练权重具有类别无关性、通用性和可迁移性。这使我们能够直接使用来自2D-3D数据集的预训练权重来启动仅3D数据集的元学习。

### 模型细节
遵循An等人 (2024) 的方法，Stratified Transformer (Lai et al., 2022) 作为我们在S3DIS和ScanNet数据集上的骨干网络，使用为S3DIS设计的Stratified Transformer架构的前两个块。IF和UF头部与相同架构的第三个块相同。来自骨干网络和两个头部的特征分别是原始点云分辨率的1/4和1/16。对于提取交互式或单模态特征，我们执行插值 (Qi et al., 2017b) 以将1/16特征从IF或UF头部上采样4倍并将它们连接到1/4骨干特征。然后，对连接的特征应用MLP以获得最终的交互式或单模态特征。单模态特征的通道维度为192，而交互式特征的维度与第一预训练步骤中使用的预训练VLMs对齐。对于LSeg (Li et al., 2022)，该维度为512，而对于OpenSeg (Ghiasi et al., 2022)，为768。遵循An等人 (2024) 的方法，来自基础数据集的输入特征包括XYZ坐标和RGB颜色。我们每个类别提取100个原型（$N_P = 100$）；对于$k > 1$的$k$-shot设置，我们从每个shot采样$N_P/k$个原型并将它们连接以获得$N_P$个原型。训练和推理在四个RTX 3090 GPU上进行。